# Balance checks
This notebook calculates water and energy balances  for all six laugh tests.

## Expectations
Balance errors should be small, relative to maginutes of states and fluxes.

No balance are calculated for the Vanderborght case, because this test case looks at a single snapshot in time and not at a time interval over which a balance can be calculated.

## Meta data

| Data  | Value  |
|:---|:---|
| Model name| Structure for Unifying Multiple Modelling Alternatives (SUMMA) |
| Model version  | See attributes in output .nc file |
| Model reference | Clark et al. (2015a,b) |
| Model runs by | R. Zolfaghari |
| Notebook code by | W. Knoben, A. Bennett |

In [7]:
# modules
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr # note, also needs netcdf4 library installed

In [147]:
# Specify the data locations relative to the notebook
file_paths = {'Celia'               : '../lt1_celia1990/output/celia1990_output_timestep.nc',
              'Miller clay'         : '../lt2_miller1998/output/millerClay_output_timestep.nc',
              'Miller loam'         : '../lt2_miller1998/output/millerLoam_output_timestep.nc',
              'Miller sand'         : '../lt2_miller1998/output/millerSand_output_timestep.nc',
              #'Vanderborght exp = 1': '../lt3_vanderborght2005/output/vanderborght2005_exp1_output_timestep.nc',
              #'Vanderborght exp = 2': '../lt3_vanderborght2005/output/vanderborght2005_exp2_output_timestep.nc',
              #'Vanderborght exp = 3': '../lt3_vanderborght2005/output/vanderborght2005_exp3_output_timestep.nc',
              'Wigmosta exp = 1'    : '../lt4_wigmosta1994/output/syntheticHillslope-exp1_output_timestep.nc',
              'Wigmosta exp = 2'    : '../lt4_wigmosta1994/output/syntheticHillslope-exp2_output_timestep.nc',
              'Colbeck exp = 1'     : '../lt5_colbeck1976/output/colbeck1976-exp1_output_timestep.nc',
              'Colbeck exp = 2'     : '../lt5_colbeck1976/output/colbeck1976-exp2_output_timestep.nc',
              'Colbeck exp = 3'     : '../lt5_colbeck1976/output/colbeck1976-exp3_output_timestep.nc',
              'Mizoguchi'           : '../lt6_mizoguchi1990/output/mizoguchi1990_output_timestep.nc'}

In [4]:
# Selection parameters
# We can use these to remove the GRU and HRU dimensions from the output files > easier data handling
GRU, HRU = 0,0

In [173]:
# function to calculate the water balance components on a given time step
def calc_wb(dat,domain):
    
    # Input: xarray dataset with variables:
    # - time 
    # - nSoil
    # - nSnow
    # - nLayers
    # - mLayerDepth
    # - mLayerVolFracLiq
    # - mLayerVolFraqIce
    # - iLayerLiqFluxSoil
    # - mLayerTranspire
    # - mLayerBaseflow
    # - mLayerCompress
    
    # Make some storage variables
    vec_liqError = []
    
    # Find the timestep size [s]
    dt = round((dat['time'][1] - dat['time'][0]).values/np.timedelta64(1, 's'))

    # Intrinsic densities (needed for snow)
    iden_liq = 1000 # [kg m-3]
    iden_ice = 917
    
    # --- Water balance components soil ---
    # Start of time loop
    for t in range(0,len(dat['time'])-1):
    
        # Specify time as indices
        S_time = t 
        E_time = t+1
    
        # Get the layer variables at t=t
        S_nSoil   = dat['nSoil'].isel(time=S_time).values
        S_nSnow   = dat['nSnow'].isel(time=S_time).values
        S_nLayers = dat['nLayers'].isel(time=S_time).values
        
        # Do computations based on the specified domain
        if domain == 'snow':
            
            # Snow layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(0,S_nSnow)).values # needs to start at index 0, not 1 in Python
            
            # Snow water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            mass0 = sum( (S_mLayerVolFraqLiq * iden_liq + S_mLayerVolFraqIce * iden_ice) * S_mLayerDepth )
            snowBalance0 = mass0 / iden_liq

            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values
            E_nSnow   = dat['nSnow'].isel(time=E_time).values
            E_nLayers = dat['nLayers'].isel(time=E_time).values
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(0,E_nSnow)).values # needs to start at index 0, not 1 in Python

            # Rainfall flux between t=t and t=t+1
            scalarRainfall = (dat['scalarRainfall'].isel(time=E_time).values / iden_liq) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Snowfall flux between t=t and t=t+1
            scalarSnowfall = (dat['scalarSnowfall'].isel(time=E_time).values / iden_ice) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Melt flux between t=t and t=t+1
            scalarRainPlusMelt = dat['scalarRainPlusMelt'].isel(time=E_time).values * dt # [m s-1] * [s] = [m]
            
            # Snow water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            mass1 = sum( (E_mLayerVolFraqLiq * iden_liq + E_mLayerVolFraqIce * iden_ice) * E_mLayerDepth )
            snowBalance1 = mass1 / iden_liq
            
            # Water balance error
            liqError = snowBalance1 - (snowBalance0 + scalarRainfall + scalarSnowfall - scalarRainPlusMelt)
            
            # Append
            vec_liqError.append(liqError)
        
        elif domain == 'soil':
        
            # Soil layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Soil water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            soilBalance0 = sum( (S_mLayerVolFraqLiq + S_mLayerVolFraqIce) * S_mLayerDepth )   
    
            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values
            E_nSnow   = dat['nSnow'].isel(time=E_time).values
            E_nLayers = dat['nLayers'].isel(time=E_time).values
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Vertical downward flux between t=t and t=t+1 (needs t=t+1)
            iLayerLiqFluxSoil_top = dat['iLayerLiqFluxSoil'].isel(time=E_time,ifcSoil=E_nSoil).values
            iLayerLiqFluxSoil_bot = dat['iLayerLiqFluxSoil'].isel(time=E_time,ifcSoil=0).values
            vertFlux = -1* (iLayerLiqFluxSoil_top - iLayerLiqFluxSoil_bot) * dt
    
            # Transpiration between t=t and t=t+1 (needs t=t+1)
            tranSink = dat['mLayerTranspire'].isel(time=E_time).sum().values*dt 
    
            # Baseflow between t=t and t=t+1 (needs t=t+1)
            baseSink = dat['mLayerBaseflow'].isel(time=E_time).sum().values*dt
    
            # Compression between t=t and t=t+1 (needs t=t+1)
            mLayerCompress = dat['mLayerCompress'].isel(time=E_time,midSoil=slice(0,E_nSoil)).values
            compSink = sum(mLayerCompress * E_mLayerDepth)
    
            # Soil water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            soilBalance1 = sum( (E_mLayerVolFraqLiq + E_mLayerVolFraqIce) * E_mLayerDepth )
    
            # Water balance error
            liqError = soilBalance1 - (soilBalance0 + vertFlux + tranSink - baseSink - compSink)
        
            # Append
            vec_liqError.append(liqError)
    
    # outputs
    return vec_liqError

Processing starts here

In [174]:
# initiate some lists
test_name = []
wbe_mean  = []
wbe_cum   = []
ebe_mean  = []
ebe_cum   = []

In [175]:
# loop over the files
for test,file in file_paths.items():
    
    # progress
    print('Working on ' + test)
    
    # load the data
    if 'Colbeck' in test:
        dat = xr.open_dataset( file ).isel(hru=HRU).load() # Colbeck output has no GRU dimensions
        domain = 'snow'
    else:
        dat = xr.open_dataset( file ).isel(hru=HRU, gru=GRU).load()
        domain = 'soil'
    
    # Get water balance values
    wbe = calc_wb(dat,domain)
    
    # Store the results
    test_name.append(test)
    wbe_mean.append(np.mean(wbe))
    wbe_cum.append(sum(wbe))

Working on Celia
Working on Miller clay
Working on Miller loam
Working on Miller sand
Working on Wigmosta exp = 1
Working on Wigmosta exp = 2
Working on Colbeck exp = 1
Working on Colbeck exp = 2
Working on Colbeck exp = 3
Working on Mizoguchi


In [176]:
# Make a dataframe
results = pd.DataFrame( {'Test'                           : test_name,
                         'Mean wbe [m]'       : wbe_mean,
                         'Cummulative wbe [m]': wbe_cum},
                       columns = ['Test', 'Mean wbe [m]', 'Cummulative wbe [m]'])
results

,Test,Mean wbe [m],Cummulative wbe [m]
0,Celia,-4.517692e-08,-5.376053e-06
1,Miller clay,-9.048453e-08,-2.153532e-05
2,Miller loam,-2.179302e-06,-5.186738e-04
3,Miller sand,-4.847109e-06,-1.153612e-03
4,Wigmosta exp = 1,-5.236198e-10,-5.267615e-07
5,Wigmosta exp = 2,-5.718102e-09,-5.752411e-06
6,Colbeck exp = 1,5.276705e-10,3.160746e-07
7,Colbeck exp = 2,1.259980e-10,7.547278e-08
8,Colbeck exp = 3,3.791698e-11,2.271227e-08
9,Mizoguchi,-4.542662e-10,-1.634904e-06
